### 1. out-of-the-box make cued widget

In [1]:
from ipywidgets import VBox
from scwidgets import (SaveRegistry, CheckRegistry, CheckRegistryDummy)

check_registry = CheckRegistryDummy()
save_registry = SaveRegistry()

class DemoWidget(VBox):
    def __init__(self, widget, trait, name, save_registry, check_registry, update_function):
        self._widget = widget
        self._trait = trait
        self._check_cue_widget = CheckCueBox(widget, trait)
        self._save_check_cue_widget = SaveCueBox(widget,  trait, widget_to_cue=self._check_cue_widget)
        self._cue_widget = UpdateCueBox(widget, trait, widget_to_cue=self._save_check_cue_widget)
        
        self._check_button = CheckCueButton(self._check_cue_widget, check_registry, self)
        self._save_button = SaveCueButton(self._save_check_cue_widget, save_registry, name)
        self._update_button = UpdateCueButton(self._cue_widget, update_function)
        super().__init__([self._cue_widget, HBox([self._update_button, self._check_button, self._save_button])])
    
        save_registry.register(self, name)
        check_registry.register_checks(self)
    
    @property
    def answer(self):
        return getattr(self._widget, trait)

    @property
    def compute_output(self):
        return self.answer

from scwidgets import (CheckCueBox, SaveCueBox, UpdateCueBox, CheckCueButton, SaveCueButton, UpdateCueButton)
from ipywidgets import VBox, HBox, FloatSlider




In [2]:
float_slider = FloatSlider(description='value')
def update_function(change=None):
    return True
# if widget does not have compute_ouput function then in uses the trait for checks
demo = DemoWidget(float_slider, "value", "demo_ex", save_registry, check_registry, update_function)
display(demo)

DemoWidget(children=(CueBox(children=(CueBox(children=(CueBox(children=(FloatSlider(value=0.0, description='va…

### 1.2 Example parbox

In [8]:
# requires portaudio19

# Linux
# $ apt install portaudio19-dev

# Mac
# $ brew instal portaudio19

In [5]:
!pip install pyaudio

  Using cached PyAudio-0.2.13.tar.gz (46 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.13-cp310-cp310-linux_x86_64.whl size=27635 sha256=99d9127568bdb19f22fd7d41ad037fb712300831c70ebc488d0532b8b25e33f2
  Stored in directory: /home/alexgo/.cache/pip/wheels/9e/3d/ab/c92aec70a8d45cb9b8feb9476ca46616cdb5c78b00dcfb0afd
Successfully built pyaudio


### 2. Customized demo

In [6]:
import math
from pyaudio import PyAudio, paUInt8

def generate_sine_wave(frequency, duration, volume=0.2, sample_rate=22050):
    ''' Generate a tone at the given frequency.

        Limited to unsigned 8-bit samples at a given sample_rate.
        The sample rate should be at least double the frequency.
    '''
    if sample_rate < (frequency * 2):
        print('Warning: sample_rate must be at least double the frequency '
              f'to accurately represent it:\n    sample_rate {sample_rate}'
              f' ≯ {frequency*2} (frequency {frequency}*2)')

    num_samples = int(sample_rate * duration)
    rest_frames = num_samples % sample_rate

    pa = PyAudio()
    stream = pa.open(
        format=paUInt8,
        channels=1,  # mono
        rate=sample_rate,
        output=True,
    )

    # make samples
    s = lambda i: volume * math.sin(2 * math.pi * frequency * i / sample_rate)
    samples = (int(s(i) * 0x7F + 0x80) for i in range(num_samples))

    # write several samples at a time
    for buf in zip( *([samples] * sample_rate) ):
        stream.write(bytes(buf))

    # fill remainder of frameset with silence
    stream.write(b'\x80' * rest_frames)

    stream.stop_stream()
    stream.close()
    pa.terminate()


In [7]:
from ipywidgets import HBox, IntSlider, HTML, Button, Output
from scwidgets import (CueButton, CheckCueBox, UpdateCueBox, CheckCueButton, UpdateCueButton)

def run_own_sound(change=None):
    with Output():
        generate_sine_wave(
            # see http://www.phy.mtu.edu/~suits/notefreqs.html
            frequency=int_slider.value,   # Hz, waves per second C6
            duration=1.0,       # seconds to play sound
            volume=0.2,        # 0..1 how loud it is
            sample_rate=22050,  # number of samples per second: 11025, 22050, 44100
        )
    return True

def run_ref_sound(change=None):
    with Output():
        generate_sine_wave(
            # see http://www.phy.mtu.edu/~suits/notefreqs.html
            frequency=515,   # Hz, waves per second C6
            duration=1.0,       # seconds to play sound
            volume=0.2,        # 0..1 how loud it is
            sample_rate=22050,  # number of samples per second: 11025, 22050, 44100
        )
    return True

def give_hint(change=None):
    hint_output.clear_output()
    with hint_output:
        if int_slider.value < 515:
            print("Too low frequency")
        elif int_slider.value > 515:
            print("Too high frequency")
        else:
            print("Correct frequency")
    return True
            
hint_output = Output() 

int_slider = IntSlider(value=500, min=500, max=520, step=1, continuous_update=False, description='frequency')
          
text = HTML(value='Can you find the right frequency?')
update_cued_int_slider = UpdateCueBox(int_slider, "value")
check_cued_int_slider = CheckCueBox(int_slider, "value", widget_to_cue=update_cued_int_slider)

run_own_sound_button = UpdateCueButton(update_cued_int_slider, run_own_sound, disable_after_succesful_action=False, description='Run your sound')

run_ref_sound_button = Button(description='Run ref sound')
run_ref_sound_button.on_click(run_ref_sound)

# check button with check registry would be overkill
# to be forced to give a css style, is overkill, if one wants to only change the color
hint_button_css_style = {'__init__': 'scwidget-check-cue-button',
             '_on_disabled': 'scwidget-check-cue-button:disabled',
             '_on_trait_change': 'scwidget-check-cue-button--on_trait_change'}

hint_button = CueButton(check_cued_int_slider, give_hint, description='Give hint', css_style=hint_button_css_style)

demo = VBox([text, check_cued_int_slider, HBox([run_own_sound_button, run_ref_sound_button, hint_button]), hint_output])
display(demo)